In [ ]:
#importing libraries
!pip install -U datasets tiktoken tqdm numpy torch matplotlib huggingface_hub
import os
import math
import time
from contextlib import nullcontext
from dataclasses import dataclass, field
from tqdm.auto import tqdm
import json
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
import tiktoken
from datasets import load_dataset
import matplotlib.pyplot as plt

In [ ]:
#configs file
from dataclasses import dataclass, field
import os
import torch
@dataclass
class SLMConfig:
    block_size: int = 256
    vocab_size: int = 50257     
    n_layer: int = 12           
    n_head: int = 12           
    n_embd: int = 768          
    dropout: float = 0.1
    bias: bool = False
    batch_size: int = 8         
    gradient_accumulation_steps: int = 4 
    max_iters: int = 100000     
    eval_interval: int = 1000   
    eval_iters: int = 200
    learning_rate: float = 3e-4 
    weight_decay: float = 0.1
    beta1: float = 0.9
    beta2: float = 0.95
    grad_clip: float = 1.0
    warmup_iters: int = 2000    
    lr_decay_iters: int = 100000
    min_lr: float = 3e-5        
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
    compile: bool = True
    data_dir: str = 'data_combined'
    num_proc: int = 8
    total_batches: int = 1024 
    out_dir: str = 'out_v3' 
    best_model_name: str = 'best_model_v3.pt'
    local_pretrained_path: str = "" 

config = SLMConfig()
config.lr_decay_iters = config.max_iters

os.makedirs(config.out_dir, exist_ok=True)
os.makedirs(config.data_dir, exist_ok=True)
best_model_path = os.path.join(config.out_dir, config.best_model_name)

ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[config.dtype]
torch.set_default_dtype(ptdtype)